# 핵심 아이디어  
한식 일식 중식이 라벨링된 데이터를 바탕으로 각 메뉴의 국적을 라벨링한다.  

한식 메뉴만 모아 뽀로로 제로샷 등으로 특정 메뉴를 라벨링한다.  

국적 라벨링 시도 

In [1]:
import pandas as pd

In [23]:
df1 = pd.read_csv('/content/MENU_INFO_KOREAN(20220930).csv', encoding='cp949')
df2 = pd.read_csv('/content/MENU_INFO_KOREAN.csv')
df3 = pd.read_csv('/content/MENU_INFO_KOREAN_seoul.csv')
df4 = pd.read_csv('/content/전라남도_메뉴정보_20210120.csv')
df5 = pd.read_csv('/content/충청북도 충주시_음식점정보_20220731.csv', encoding='cp949')

In [26]:
df5.head()

,업종명,업소명,소재지(도로명),소재지전화,업태명,문의전화
0,일반음식점,아서원,충청북도 충주시 관아1길 8-1 (성내동),043-847-3639,중국식,043-850-3473
1,일반음식점,중원생고기마을,충청북도 충주시 관아3길 11 (성서동),043-847-1525,식육(숯불구이),043-850-3473
2,일반음식점,대동식당,충청북도 충주시 수안보면 온천중앙길 23-1,043-846-3406,한식,043-850-3473
3,일반음식점,유선분식,충청북도 충주시 예성로 168 (성서동),043-847-2003,분식,043-850-3473
4,일반음식점,만리식당,충청북도 충주시 수안보면 물탕2길 5,043-846-3206,한식,043-850-3473


유일하게 업태가 존재하는 df5이지만 가장 중요한 메뉴명이 없어서 실패  

다음 아이디어1: '국밥' 처럼 메뉴명이 명확한 메뉴 위주로 라벨링, 메뉴명은 빈도수 키워드 분석으로.  
즉, 빈도가 어느정도 있는 메뉴는 '표준화된' 메뉴명일 확률이 높다는 전제를 이용하는 아이디어  
  
다음 아이디어2: 빈도수가 어느정도 있는 키워드를 전처리해서 깔끔하게 만든 후, 유사도 분석하여 클러스터링... 흠...  

아이디어1 구현  

In [68]:
!pip install --upgrade pip
!pip install konlpy
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 kB 40.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6688 sha256=9f6499f7e9290ef32d7fb174a9e209d376d7acc780c0be85892f9ffe2719c8e5
  Stored in directory: /root/.cache/pip/wheels/6a/31/aa/30d72ec285d8edd5283cf950df190f76a2594cf39ef6be3453
Successfully built hgt

In [125]:
import nltk
from konlpy.tag import Okt
from tqdm import tqdm
import re

In [ ]:
#1, 최빈값으로 키워드 검색해보기
t = Okt()

# 명사만 추출
menu = df['메뉴명'].tolist()
menu
# menu = [t.nouns(ch) for ch in tqdm(menu)]

In [97]:
# 명사들 빈도 파악
count_dict = {}
for i in tqdm(menu):
  if i not in count_dict:
    count_dict[i] = 1
  else:
    count_dict[i] += 1

100%|██████████| 194822/194822 [00:00<00:00, 1372924.50it/s]


In [99]:
# 빈도를 파악하여 일정 빈도수 이하의 단어들을 모두 거른다 
items = count_dict.items()
sorted_items = sorted(items, key=lambda x: x[1], reverse=True) # 내림차순 정렬렬

# cut_line = int(len(sorted_items) / 100) # 빈도 상위 1퍼센트의 단어만 선택
# outliner_vocab = [k for k, v in sorted_items[:cut_line]]

In [ ]:
sorted_items[:100]

In [102]:
df = pd.read_csv('/content/menu_concated_v1.2', index_col=0)

In [103]:
df['category'] = None

In [127]:
# outliner_vocab와 같은 메뉴명을 가졌다면 카테고리를 메뉴명으로 설정정
for k, v in sorted_items[:100]:
  k_re = re.sub('[0-9a-zA-Z()]', '', k)
  df.loc[df[df['메뉴명'].isin([k])].index, 'category'] = k_re

In [128]:
df[df['메뉴명'].isin(['바닐라라떼(HOT)'])].head()

,식당명,메뉴명,메뉴가격,category
index,,,,
3279,문토스트,바닐라라떼(HOT),4000,바닐라라떼
4635,해오라기커피,바닐라라떼(HOT),5000,바닐라라떼
17665,홍미당,바닐라라떼(HOT),4800,바닐라라떼
18808,파리바게뜨,바닐라라떼(HOT),4000,바닐라라떼
28709,만달레이,바닐라라떼(HOT),5000,바닐라라떼


In [129]:
df[df['메뉴명'].isin(['바닐라라떼'])].head()

,식당명,메뉴명,메뉴가격,category
index,,,,
238,카페051,바닐라라떼,3000,바닐라라떼
345,그숲,바닐라라떼,5000,바닐라라떼
588,오드이븐,바닐라라떼,5000,바닐라라떼
2769,메종카페,바닐라라떼,3000,바닐라라떼
6163,달맞이휴게소,바닐라라떼,6500,바닐라라떼


In [130]:
df[df['메뉴명'].isin(['아메리카노'])].head()

,식당명,메뉴명,메뉴가격,category
index,,,,
209,할리스커피,아메리카노,4100,아메리카노
223,세컨하우스,아메리카노,4500,아메리카노
236,카페051,아메리카노,1500,아메리카노
246,우즈커피,아메리카노,3500,아메리카노
340,그숲,아메리카노,4000,아메리카노


In [133]:
df[df['메뉴명'].isin(['아메리카노(HOT)'])].head()

,식당명,메뉴명,메뉴가격,category
index,,,,
3269,문토스트,아메리카노(HOT),2500,아메리카노
4633,해오라기커피,아메리카노(HOT),4000,아메리카노
4802,카페 브론즈 웨인,아메리카노(HOT),5000,아메리카노
7691,조선델리,아메리카노(HOT),5500,아메리카노
17661,홍미당,아메리카노(HOT),3800,아메리카노


In [134]:
df[df['메뉴명'].isin(['아이스아메리카노'])].head()

,식당명,메뉴명,메뉴가격,category
index,,,,
43341,할리스커피,아이스아메리카노,1800,None
46194,엘리팝,아이스아메리카노,3800,None
50513,도담도담식빵,아이스아메리카노,1500,None
53160,커피에반하다,아이스아메리카노,1500,None
66840,치악산밤나무집,아이스아메리카노,3000,None


나쁜 접근은 아닌 듯 하지만 아무래도 세세한 개입이 필요할 것 같다. 아이스 아메리카노를 아메리카노로 분류하지 않는 이유는 뭐지? isin 메소드가 내 생각과 약간 다르게 작동하네.  